# 池化层

## 1. 池化层nn.MaxPool & nn.AvgPool

无论是调镇步长还是加入填充，我们都希望能够自由控制特征图的尺寸。除了卷积之外，另一种可以高效减小特征图尺寸的操作是"池化"Pooling。池化是一种非常简单（甚至有些粗暴的）的降维方式。经常跟在卷积层之后，用以处理特征图。最常见的是最大池化（Max Pooling）和平均池化（Average Pooling）两种操作，他们都很容易理解：

![Alt text](image-118.png)

池化层也有核，但它的核没有值，只有尺寸。在上图之中，池化核的尺寸就是（2, 2）。池化核的移动也有步长stride，但默认步长就等与它的核尺寸，这样可以保证它在扫描特征图时不出现重叠。当然，如果我们需要，我们也可以设置参数令池化核的移动步长不等于核尺寸，在行业中这个叫"Overlapping Pooling",即重叠池化，但它不是非常常见。通常来说，对于特征图中每一个不重叠的，大小固定的矩阵，池化核都按照池化标准对数字进行计算或筛选。在最大池化中，它选出扫描区域中最大的数字。在平均池化中，他对扫描区域中所有的数字求平均。在加和池化中，它对扫描区域中所有的数字进行加和。

在这几种简单的方法中，最大池化是应用最广泛的，也是最有效的。考虑看看feature map中的信息是怎么得来的？feature map 中每个像素上的信息都是之前卷积层中，图像与卷积核进行互相关操作的结果。对之前的卷积层而言，卷积核都是一致的，唯一不同就是每次被扫描的区域中的像素值。像素值越大，说明颜色信息越多，像素值越小，说明图像显示约接近黑色，因此经过卷积层之后，像素值更高的点在原始图像上更有可能带有大量信息，MaxPooling通过摘取这些带有更多信息的像素点，有效将冗余信息消除，实现了特征的提炼。相对的，平均和加和的提炼效应就弱一些。

在PyTorch中，池化层也有多种选项，但这些多属于“普通池化”的范围。在传统计算机视觉中，我们还有空间金字塔池化等操作。

![Alt text](image-119.png)

以MaxPool2d为例，其类和参数的详情如下：

In [ ]:
# MaxPool2d参数
CLASS torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)


# kernel_size：池化窗口大小
# stride：步长
# padding：填充
# dilation：扩张
# return_indices：是否返回最大值索引，一般不用
# ceil_mode：是否向上取整



其中kernel_size就是池化核的尺寸，一般都设置为2*2，或者是3*3。Padding参数与Stride参数一般都不填写。需要提醒的是，池化层的步长一般与核尺寸保持一致。因此stride参数的默认值就是kernel_size。池化层对特征图尺寸的影响，也符合我们之前所写的这个公式：

![Alt text](image-120.png)

只不过此时的padding，kernel_size以及stride都是池化层的参数。我们在代码中来看看：

In [1]:
import torch
from torch import nn

data = torch.ones(size= (10, 3, 28, 28))

conv1 = nn.Conv2d(3, 6, 3)
conv3 = nn.Conv2d(6, 16, 5, stride=2, padding=1)

pool1 = nn.MaxPool2d(2)

pool1(conv3(conv1(data))).shape


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

torch.Size([10, 16, 6, 6])

除了能够有效降低模型所需的计算量，去除冗余信息之外，池化层还有什么特点和作用呢？

1. 提供非线性变化。卷积层的操作虽然复杂，但本质还是线性变化，所以我们通常会在卷积层的后面增加激活层，提供ReLu等非线性激活函数的位置。但池化层自身就是一种非线性变化，可以为模型带来一些活力。然而，学术界一直就池化层的存在必要性争论不休，因为有众多研究表明池化层并不能提升模型效果。

2. 有一定的平移性。

3. 池化层所有的参数都是超参数，不涉及到任何可以学习的参数，这即是优点（不会增加参数量），也是致命问题（池化层不会随着算法一起进步）。

4. 按照所有的规律对所有的feature map一次性进行降噪，feature map不同其本质规律不然不同，使用同样的规则进行降维，必然引起大规模信息损失。

不过在经典神经网络架构中，池化层依然是非常关键的存在。如果感兴趣的化，可以就池化与卷积的交互相应深入研究下去，继续探索提升神经网络效果的可能性。

## 2. Dropout2d与BatchNorm2d

Dropout与BN是神经网络中非常经典的，用于控制过拟合，提升模型性能泛化能力的技巧，在卷积神经网络中我们需要应用的是二维Dropout与二维BN。对于BN我们在前面的课程中有深入的研究，它是对数据进行归一化处理的经典方法，对于图像数据，我们所需要的类如下：

CLASS torch.nn.BatchNorm2d(num_features, eps, momentum, affine, track_running_stats)

BN2d所需要的输入数据是四维数据（第一个维度是samples），我们需要填写的参数几乎只有num_features一个。在处理表数据的BatchNorm1d里，num_features代表了输入bn层的神经元个数，然而对于卷积网络来说，由于存在参数共享机制，则必须以卷积核/特征图为单位来进行归一化，因此当出现在卷积网络前后时，BatchNorm2d所需输入的是上一层输出的特征图的数量。例如：

In [ ]:
data = torch.ones(size= (10, 3, 28, 28))
conv1 = nn.Conv2d(3, 32, 5, padding=2)
bn1 = nn.BatchNorm2d(32)

同时，BN层带有$\beta$ 和 $\gamma$ 参数，这两个参数的数量也由特征图的数量决定。例如，对有32张特征图的数据进行归一化时，就需要使用32组不同的$\gamma$和$\beta$参数，总参量为特征图数*2 = 64

Dropout是课程中首次提到的概念，它是指在神经网络训练过程中，以概率p随机地沉默一部分神经元的技术。具体来说，当整体神经元数量为N时，Droupout层会随机选择p*N个神经元，让这些神经元在这一次训练中不再有效，使选出的神经元的权重变为0,使得神经元失活。在每次训练中，都有一组随机挑选的神经元被沉默，这样会减弱全体神经元之间的联合适应性，减少过拟合的可能性。在进行测试时，dropout会对所有神经元上的系数都乘以概率p，用以模拟在训练中这些神经元只有P的概率被用于向前传播的状况。

![Alt text](image-121.png)

对于卷积神经网络来说，我们需要使用的类是Dropout2d，唯一需要输出的参数是p，其输入数据同样是带有samples维度的四维数据。不过在卷积中，Dropout不会以神经元为单位执行“沉默”，二是一次性毙掉一个通道。因此，当通道总数不多时，使用Dropout或Dropout中的p值太大都会让CNN丧失学习能力，造成过拟合。通常来说，使用Dropout之后模型需要更多的迭代才能够收敛，所以我们总是从P = 0.1, 0.25开始尝试，最多使用p = 0.5，否则模型的学习能力会出现明显下降。

CLASS torch.nn.Dropout2d（p = 0.5, inplace = False）

In [2]:
data = torch.ones(size= (10, 1, 28, 28))

conv1 = nn.Conv2d(1, 32, 5, padding=2)
dp1 = nn.Dropout2d(0.5)

dp1(conv1(data)).shape

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

torch.Size([10, 32, 28, 28])

Dropout层本身不带有任何需要学习的参数，因此不会影响参数量。

接下来，我们来实现一些由卷积层和池化层组成的神经网络架构，帮助大家回顾一下神经网络的定义过程，同时加深对卷积，池化等概念的印象。